In [1]:
from Plain import Dissertation_Plain_1D

In [2]:
# ---- Imports you likely already have ----
import os, json
from pathlib import Path
import numpy as np
from joblib import Parallel, delayed

In [4]:
# ==== Helpers for saving ====
def _ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def _atomic_write_json(path: Path, obj):
    tmp = path.with_suffix(".tmp")
    with open(tmp, "w") as f:
        json.dump(obj, f, indent=2)
    os.replace(tmp, path)

def _save_summary(run_dir: Path, meta: dict):
    with open(run_dir / "summary.json", "w") as f:
        json.dump(meta, f, indent=2)

def _save_fronts(run_dir: Path, t_fronts, x_fronts):
    # Backward-compatible: saves the default (N) fronts
    np.savez_compressed(run_dir / "fronts.npz",
                        t_fronts=np.asarray(t_fronts),
                        x_fronts=np.asarray(x_fronts))

def _save_fronts_named(run_dir: Path, name: str, t_fronts, x_fronts):
    np.savez_compressed(run_dir / f"fronts_{name}.npz",
                        t_fronts=np.asarray(t_fronts),
                        x_fronts=np.asarray(x_fronts))

def _save_snapshots_every_150(run_dir: Path, model):
    # every 150th step, plus the final step
    idx = np.unique(np.concatenate([
        np.arange(0, model.Nt, 150),
        np.array([model.Nt - 1])
    ]))
    np.savez_compressed(
        run_dir / "snapshots.npz",
        x=model.x,
        times=model.times[idx],
        N_arr=model.N_arr[idx, :],
        M_arr=model.M_arr[idx, :]
    )

# ==== Worker with failure logging (now stores N and M speeds) ====
def run_one(lam, m0,
            base_dir="speeds_plain",
            model_kwargs=None,
            overwrite=False):
    if model_kwargs is None:
        model_kwargs = {}

    try:
        base = Path(base_dir)
        lam_dir = base / f"lambda_{lam}"
        run_dir = lam_dir / f"m0_{m0}"
        _ensure_dir(run_dir)

        # Skip if exists (unless overwrite)
        if not overwrite and (run_dir / "summary.json").exists():
            return ("skipped", lam, m0)

        # Avoid thread over-subscription
        os.environ.setdefault("OMP_NUM_THREADS", "1")
        os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
        os.environ.setdefault("MKL_NUM_THREADS", "1")
        os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

        # Build and run model
        model = Dissertation_Plain_1D(k=lam, m0=m0, **model_kwargs)
        model.solve()

        # =========================
        # N-front (tumour) metrics
        # =========================
        c_N, b_N, r2_N = model.estimate_wave_speed(
            threshold=0.5, band=(0.1, 0.9), spline_type='cubic',
            plot=False, target='N'
        )
        # Treat missing/NaN speed as a failure
        if (c_N is None) or (isinstance(c_N, float) and np.isnan(c_N)):
            raise ValueError("Wave speed (N) could not be calculated")
        model.wave_speed = c_N  # keep your convention

        # Front tracking for N
        tN, xN = model.track_wavefront_local_interpolation(
            threshold=0.5, band=(0.1, 0.9),
            spline_type='cubic', target='N'
        )

        # =========================
        # M-front (ECM) metrics
        # =========================
        # Use m0/2 as threshold; if m0<=0, skip M speed tracking
        m_threshold = 0.5 * float(m0)
        c_M, r2_M, tM, xM = None, None, None, None
        if m_threshold > 0.0:
            try:
                c_M, b_M, r2_M = model.estimate_wave_speed(
                    threshold=m_threshold, band=(0.1, 0.9),
                    spline_type='cubic', plot=False, target='M'
                )
                # Track M front only if speed was computed
                if (c_M is not None) and not (isinstance(c_M, float) and np.isnan(c_M)):
                    tM, xM = model.track_wavefront_local_interpolation(
                        threshold=m_threshold, band=(0.1, 0.9),
                        spline_type='cubic', target='M'
                    )
            except Exception:
                # Leave M metrics as None; do not fail the whole run
                pass

        # Save artifacts
        summary = dict(
            lambda_val=lam, m0=m0,
            # N-front
            wave_speed=float(c_N),
            r2=(float(r2_N) if r2_N is not None else None),
            # M-front
            m_threshold=m_threshold,
            m_wave_speed=(float(c_M) if (c_M is not None and not (isinstance(c_M, float) and np.isnan(c_M))) else None),
            m_r2=(float(r2_M) if r2_M is not None else None),
            # meta
            dt=model.dt, T=model.T, L=model.L, N=model.N,
            init_type=model.init_type,
            t_start=model.t_start, t_end=model.t_end,
            saved_stride=150
        )
        _save_summary(run_dir, summary)

        # Save fronts (keep original fronts.npz for N; also named files)
        _save_fronts(run_dir, tN, xN)
        _save_fronts_named(run_dir, "N", tN, xN)
        if (tM is not None) and (xM is not None):
            _save_fronts_named(run_dir, "M", tM, xM)

        # Snapshots unchanged
        _save_snapshots_every_150(run_dir, model)

        # Return both N and M so the caller can flag low-R² cases
        return ("done", lam, m0,
                float(c_N), (float(r2_N) if r2_N is not None else None),
                (float(c_M) if c_M is not None else None),
                (float(r2_M) if r2_M is not None else None))

    except Exception as e:
        return ("failed", lam, m0, str(e))

# ==== Parallel grid runner with failure list AND low-R² lists for N & M ====
def run_grid(lambda_vals, m0_vals,
             base_dir="speeds_plain",
             model_kwargs=None,
             overwrite=False,
             n_jobs=-1, verbose=10):

    if model_kwargs is None:
        model_kwargs = {}

    tasks = [(lam, m0) for lam in lambda_vals for m0 in m0_vals]
    results = Parallel(n_jobs=n_jobs, verbose=verbose, backend="loky")(
        delayed(run_one)(
            lam, m0,
            base_dir=base_dir,
            model_kwargs=model_kwargs,
            overwrite=overwrite
        ) for lam, m0 in tasks
    )

    done, skipped, failed, low_r2_N, low_r2_M = [], [], [], [], []
    for r in results:
        tag = r[0]
        if tag == "done":
            # r = ("done", lam, m0, cN, r2N, cM, r2M)
            _, lam, m0, cN, r2N, cM, r2M = r
            done.append({"lambda": lam, "m0": m0, "cN": cN, "r2N": r2N, "cM": cM, "r2M": r2M})
            # Flag low-R² (strictly less than 1.0) for N
            if (r2N is None) or (isinstance(r2N, float) and (np.isnan(r2N) or r2N < .999)):
                low_r2_N.append({"lambda": lam, "m0": m0, "cN": cN, "r2N": r2N})
            # Flag low-R² for M only if we attempted it (cM not None)
            if (cM is not None):
                if (r2M is None) or (isinstance(r2M, float) and (np.isnan(r2M) or r2M < .999)):
                    low_r2_M.append({"lambda": lam, "m0": m0, "cM": cM, "r2M": r2M})
        elif tag == "skipped":
            _, lam, m0 = r
            skipped.append({"lambda": lam, "m0": m0})
        elif tag == "failed":
            _, lam, m0, msg = r
            failed.append({"lambda": lam, "m0": m0, "error": msg})

    # Save failure and low-R² lists for later
    base = Path(base_dir)
    _ensure_dir(base)
    _atomic_write_json(base / "failed_runs.json", failed)
    _atomic_write_json(base / "low_r2_runs_N.json", low_r2_N)
    _atomic_write_json(base / "low_r2_runs_M.json", low_r2_M)

    # Console summary
    print(f"✅ Done: {len(done)}, Skipped: {len(skipped)}, Failed: {len(failed)}, Low-R² (N): {len(low_r2_N)}, Low-R² (M): {len(low_r2_M)}")
    if failed:
        print("❌ Failed runs:")
        for item in failed:
            print(f"  lambda={item['lambda']}, m0={item['m0']} | error: {item['error']}")
    if low_r2_N:
        print("⚠️  Low-R² runs for N (R² < 1):")
        for item in low_r2_N[:10]:  # show first 10 for brevity
            print(f"  lambda={item['lambda']}, m0={item['m0']} | cN={item['cN']} | r2N={item['r2N']}")
    if low_r2_M:
        print("⚠️  Low-R² runs for M (R² < 1):")
        for item in low_r2_M[:10]:
            print(f"  lambda={item['lambda']}, m0={item['m0']} | cM={item['cM']} | r2M={item['r2M']}")

    return {"done": done, "skipped": skipped, "failed": failed,
            "low_r2_N": low_r2_N, "low_r2_M": low_r2_M}

# =========================
# ===== RUN THE BATCHS =====
# =========================

# ---- Original batch (kept exactly as requested) ----
lambda_vals = [0.001, 0.01, 0.05, 1, 5, 10]

m0_vals = [0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5,
           0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]

shared_kwargs = dict(
    L=200, N=5001, T=200, dt=0.1,
    init_type="tanh", steepness=0.85, perc=0.4,
    t_start=40, t_end=180, num_points=200,
    n0=1.0, K=1.0, rho=1.0, D=1.0, Mmax=1.0
)

results = run_grid(lambda_vals, m0_vals,
                   base_dir="speeds_plain",
                   model_kwargs=shared_kwargs,
                   overwrite=False,
                   n_jobs=-1, verbose=10)

# ---- Additional run for lambda = 100 with DIFFERENT L/N/T/dt/time-window ----
lambda_vals_100 = [100]
shared_kwargs_100 = dict(
    L=50, N=5001, T=100, dt=0.01,
    init_type="tanh", steepness=0.85, perc=0.4,
    t_start=20, t_end=31, num_points=200,
    n0=1.0, K=1.0, rho=1.0, D=1.0, Mmax=1.0
)

results_100 = run_grid(lambda_vals_100, m0_vals,
                       base_dir="speeds_plain",
                       model_kwargs=shared_kwargs_100,
                       overwrite=False,
                       n_jobs=-1, verbose=10)

# ==== Print flagged results (checks BOTH N and M) ====
print("\n=== Summary of Problematic Runs (N & M) ===")
flagged = []

def _flag_from_summary(meta, lam, m0):
    # N-front checks
    cN = meta.get("wave_speed", None)
    r2N = meta.get("r2", None)
    if cN is None:
        flagged.append({"lambda": lam, "m0": m0, "reason": "NO N-SPEED"})
    elif (r2N is None) or (isinstance(r2N, float) and (np.isnan(r2N) or r2N < 0.999)):
        flagged.append({"lambda": lam, "m0": m0, "reason": f"LOW N R² ({r2N:.4f})" if r2N is not None else "LOW N R² (None)"})
    # M-front checks (only if reported)
    cM = meta.get("m_wave_speed", None)
    r2M = meta.get("m_r2", None)
    mthr = meta.get("m_threshold", None)
    if mthr is not None and mthr > 0:
        if cM is None:
            flagged.append({"lambda": lam, "m0": m0, "reason": "NO M-SPEED"})
        elif (r2M is None) or (isinstance(r2M, float) and (np.isnan(r2M) or r2M < 0.999)):
            flagged.append({"lambda": lam, "m0": m0, "reason": f"LOW M R² ({r2M:.4f})" if r2M is not None else "LOW M R² (None)"})

# Combine both batches’ “done” entries for quick iteration
all_done = results["done"] + results_100["done"]
for d in all_done:
    lam, m0 = d["lambda"], d["m0"]
    summary_path = Path("speeds_plain") / f"lambda_{lam}" / f"m0_{m0}" / "summary.json"
    if summary_path.exists():
        with open(summary_path, "r") as f:
            meta = json.load(f)
        _flag_from_summary(meta, lam, m0)

# Print list
if flagged:
    for item in flagged:
        print(f"λ={item['lambda']}, m0={item['m0']} -> {item['reason']}")
else:
    print("✅ No failures, no low R² (N or M), and all available speeds calculated.")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


❌ Not enough valid front points.
Estimated speed = 0.6123, R² = 0.9997
Estimated speed = 0.8141, R² = 0.9999
Estimated speed = 0.2062, R² = 0.9992
Estimated speed = 0.9256, R² = 1.0000
❌ Not enough valid front points.
❌ Not enough valid front points.
Estimated speed = 0.9838, R² = 1.0000


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   32.4s


❌ Not enough valid front points.
❌ Not enough valid front points.
Estimated speed = 1.0023, R² = 1.0000
❌ Not enough valid front points.
❌ Not enough valid front points.
Estimated speed = 0.4475, R² = 0.9995
❌ Not enough valid front points.
Estimated speed = 0.9838, R² = 1.0000
❌ Not enough valid front points.


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   54.6s


...

Estimated speed = 0.7665, R² = 1.0000
Estimated speed = 0.9962, R² = 1.0000
Estimated speed = 0.6238, R² = 1.0000
Estimated speed = 1.0034, R² = 0.9997


[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:  4.0min remaining:  1.0min


Estimated speed = 0.6091, R² = 1.0000
Estimated speed = 0.7063, R² = 1.0000
Estimated speed = 0.5966, R² = 0.9998
Estimated speed = 0.6997, R² = 1.0000


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  4.1min finished


✅ Done: 15, Skipped: 0, Failed: 0, Low-R² (N): 0, Low-R² (M): 0

=== Summary of Problematic Runs (N & M) ===
λ=0.001, m0=0.01 -> NO M-SPEED
λ=0.001, m0=0.05 -> NO M-SPEED
λ=0.001, m0=0.1 -> NO M-SPEED
λ=0.001, m0=0.2 -> NO M-SPEED
λ=0.001, m0=0.3 -> NO M-SPEED
λ=0.001, m0=0.4 -> NO M-SPEED
λ=0.001, m0=0.5 -> NO M-SPEED
λ=0.001, m0=0.6 -> NO M-SPEED
λ=0.001, m0=0.7 -> NO M-SPEED
λ=0.001, m0=0.8 -> NO M-SPEED
λ=0.001, m0=0.9 -> NO M-SPEED
λ=0.001, m0=0.95 -> NO M-SPEED
λ=0.001, m0=0.99 -> NO M-SPEED
λ=0.01, m0=0.01 -> NO M-SPEED
λ=0.01, m0=0.05 -> NO M-SPEED
λ=0.01, m0=0.1 -> NO M-SPEED
λ=0.01, m0=0.2 -> NO M-SPEED
λ=0.01, m0=0.9 -> LOW M R² (0.9982)
λ=0.01, m0=0.95 -> LOW M R² (0.9915)
λ=0.01, m0=0.99 -> LOW M R² (0.9366)
λ=0.01, m0=1 -> LOW M R² (0.7443)
λ=0.05, m0=0.01 -> NO M-SPEED
λ=0.05, m0=0.05 -> NO M-SPEED
λ=0.05, m0=0.1 -> NO M-SPEED
λ=0.05, m0=0.2 -> NO M-SPEED
λ=0.05, m0=0.95 -> LOW M R² (0.9977)
λ=0.05, m0=0.99 -> LOW M R² (0.9863)
λ=0.05, m0=1 -> LOW M R² (0.9494)
λ=1, m0=0